In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [2]:
# Ubah nilai sentimen
def map_sentiment(sentiment):
    if sentiment == 1:
        return 'positif'
    elif sentiment == 0:
        return 'netral'
    else:
        return 'negatif'

df = pd.read_csv('pre_title_steaming.csv')
df['sentimen'] = df['sentimen'].map(map_sentiment)

In [3]:
df['sentimen'] 

0        positif
1        positif
2        positif
3        positif
4        positif
          ...   
14995     netral
14996     netral
14997     netral
14998     netral
14999     netral
Name: sentimen, Length: 15000, dtype: object

In [4]:
len(df)

15000

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['title'], df['sentimen'], test_size=0.3, random_state=42)

In [6]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [7]:
# Buat model klasifikasi Naive Bayes
nb_classifier = MultinomialNB()

# Latih model
nb_classifier.fit(X_train_tfidf, y_train)

# Lakukan prediksi
y_pred = nb_classifier.predict(X_test_tfidf)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi model Naive Bayes: {:.2f}%".format(accuracy * 100))

Akurasi model Naive Bayes: 62.29%


# Grid search

In [8]:
from sklearn.model_selection import GridSearchCV

# Definisikan ruang hyperparameter yang akan dijelajahi
param_grid = {
    'alpha': [0.5, 1.0, 1.5],  # Variasi smoothing parameter
}